In [ ]:
!pip install --user scprep phate scanpy anndata

In [ ]:
import scprep

In [ ]:
scprep.io.download.download_google_drive('1VuddKA7Xa23NAFKvRUmtuND16eEMHL9z', '/content/r_packages.tar.gz')
!tar xzf /content/r_packages.tar.gz && rm /content/r_packages.tar.gz
!apt-get install -yqq libgsl-dev=2.4+dfsg-6
!pip install -U rpy2
!echo "R_LIBS_USER='/content/.r_packages'" >> ~/.Renviron
!R -e "BiocManager::install('splatter')"

In [ ]:
import pandas as pd
import numpy as np
import phate

import matplotlib.pyplot as plt

import os, tarfile

import anndata, scanpy
# matplotlib settings for Jupyter notebooks only
%matplotlib inline

# Comparing trajectory inference algorithms on Treutlein fibroblast data

In this exercise, we will run both Diffusion Pseudotime and Slingshot on a dataset of 392 cells generated from trans-differentiating fibroblasts from [Treutlein et al. _Nature_. 2016](http://doi.org/10.1038/nature18323). This data was already preprocessed and downloaded from the [Dynverse quickstart tutorial](https://dynverse.org/users/2-quick_start/).

Briefly, a variety of transcription factors were overexpressed in Mouse Embryonic Fibroblasts (MEFs) to induce neuronal differentiation. Cells were sampled at several intervals over 22 days of differentation and sequenced using the Fluidigm C1 system and the SMART-seq protocol.

Here, we're doing to work with data that has already been filtered, preprocessed, and normalized so we can get straight to work running trajectory inference algorithms. This preprocessing includes selection of 2000 highly variable genes so the data matrix is 392 cells by 2000 genes.

Unlike the previous exercise, we're going to use the Diffusion Psuedotime (DPT) and Slingshot implementations provided by the original authors of each paper. DPT is provided as part of the excellent [Scanpy](https://scanpy.readthedocs.io/en/stable/) single cell analysis toolkit. Slingshot is only available from the R implementation, so we provided a convenience wrapper in `scprep.run`.

We're breaking this section into the following parts. First, we'll run Slingshot on the Treutlein data. Next, we'll run diffusion pseudotime and compare the results.

## Loading and visualizing the data

#### Download data from Google Drive

In [ ]:
download_dir = os.path.expanduser('~')
tar_path = os.path.join(download_dir, '2014_Treutlein_Fibroblast.tar.gz')

# Download the data tarball
if not os.path.exists(tar_path):
    scprep.io.download.download_google_drive('1WzEva0PM95an9hKiwgQydV_df0P_g63Q', tar_path)

# Extract data directory
with tarfile.open(tar_path) as tf:
    tf.extractall(download_dir)

# Format the data paths
data_dir = os.path.join(download_dir, '2014_Treutlein_Fibroblast_induced_neural_differentation')

data_path = os.path.join(data_dir, 'Treutlein.expression.csv')
metadata_path  = os.path.join(data_dir, 'Treutlein.groups.csv')

#### Read in the data using `scprep`

In [ ]:
data = scprep.io.load_csv(data_path)
data.head()

In [ ]:
# we can read in arbitrary csv files with scprep
# here the columns will be metadata features rather than genes
metadata = scprep.io.load_csv(metadata_path, gene_names=['annotated_clusters'], skiprows=1)
metadata.head()

#### Run PHATE

In [ ]:
phate_op = phate.PHATE()

data_phate = phate_op.fit_transform(data)
data_phate = pd.DataFrame(data_phate, index=data.index)

In [ ]:
scprep.plot.scatter2d(data_phate, c=metadata['annotated_clusters'],
                      legend_anchor=(1,1),
                      label_prefix='PHATE', ticks=None,
                      title='Annotated Clusters',
                      figsize=(7,5))

## Running Slingshot

[Slingshot](https://doi.org/10.1186/s12864-018-4772-0) is a trajectory inference tool that was reported to be among the top-performing trajectory inference algorithms for simple trajectories. The method has five conceptual steps:

1. Perform dimensionality reduction on the data (e.g. PCA, PHATE, UMAP)
2. Cluster the data
3. Create a minimum spanning tree between the data
4. Fit principal curves to the data
5. Project the data onto the principal curves



### Step 1 - Perform dimensionality reduction

We already did this! For now, let's proceed to Step 2.

**Bonus** If you have extra time later, try using a different dimensionality reduction algorithm by removing the comments at the beginning of the next code block.

In [ ]:
# =================
# To run the next two lines, remove the hashtags at the beginning of each line
# data_umap = umap.UMAP().fit_transform(data)
# data_pca = scprep.reduce.pca(data, n_components=2)
# =================

### Step 2 - Cluster the data

Picking the ideal number of clusters here is a little trial and error. Let's try a few different number of clusters and compare.

In [ ]:
fig, axes = plt.subplots(1,4, figsize=(16,4))

ks = [3,4,5,6]
for ax, k in zip(axes, ks):
    # Run PHATE clustering
    clusters = phate.cluster.kmeans(phate_op, n_clusters=k)
    metadata['cluster'] = clusters
    scprep.plot.scatter2d(data_phate, c=metadata['cluster'],
                     title='n_clusters = {}'.format(k),
                     label_prefix='PHATE', ticks=None,
                     ax=ax)
    
fig.tight_layout()

#### Pick a number of clusters that looks good to you


In [ ]:
# =========
# Set the number of clusters here
clusters = phate.cluster.kmeans(phate_op, n_clusters= ?)
metadata['cluster'] = clusters
# =========

In [ ]:
scprep.plot.scatter2d(data_phate, c=metadata['cluster'],
                 label_prefix='PHATE', ticks=None,
                figsize=(5,5))

### Step 3, 4, 5 - Create the minimum spanning tree, fit principal curves, and project data onto curves

These last steps of Slingshot are all performed behind-the-scenes by the Slingshot R package. We're going to leave all of those steps to Slingshot here and access the method using `scprep.run.Slingshot`. You can find the full documentation for the method here: https://scprep.readthedocs.io/en/stable/reference.html#module-scprep.run

#### Run `scprep.run.Slingshot()`

In [ ]:
# =======
# Reading the documentation for `scprep.run.Slingshot()` 
# replace the '...' with the correct code.
results = scprep.run.Slingshot( ... )

# =======

#### Grab output from `results` dictionary

In [ ]:
pseudotime = results['pseudotime']
metadata['slingshot1'] = pseudotime.iloc[:,0]
metadata['slingshot2'] = pseudotime.iloc[:,1]

metadata['slingshot_branch'] = results['branch']

#### Plotting the principal curves


In [ ]:
ax = scprep.plot.scatter2d(data_phate, c=metadata['slingshot_branch'],
                 label_prefix='PHATE', ticks=None,
                 title='Slingshot Principal Curves',
                 figsize=(5,5))

# `results['curves']` returns as tensor (an array of arrays)
# Each array contains coordinates for the curves in the dimensionality reduced space
for curve in results['curves']:
    ax.plot(curve[:,0], curve[:,1], c='k', linewidth=3)

### Plotting branch pseudotime

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(11,5))

for i, ax in enumerate(axes):
    curr_pseudotime = metadata['slingshot{}'.format(i+1)]
    m = np.isfinite(curr_pseudotime)
    # plot null points as grey
    scprep.plot.scatter2d(data_phate[~m], color='grey', s=10, ax=ax)    
    # plot points on branch
    scprep.plot.scatter2d(data_phate[m], c=curr_pseudotime[m], s=10,
                          label_prefix='PHATE', ticks=None,
                          title='Branch {}'.format(i + 1), ax=ax)
    
fig.tight_layout()

### Plotting clusters by pseudotime

Now we can do something interesting, like figure out the developmental ordering of the various cell types characterized in the `annotated_clusters` metadata variable.

To plot clusters by pseudotime, we're going to use `scprep.plot.jitter()`. Jitterplots are a kind of scatter plot useful for comparing the distribution of values within various classes of data. They are related to [`sns.violinplot`](https://seaborn.pydata.org/generated/seaborn.violinplot.html) and [`sns.swarmplot`](https://seaborn.pydata.org/generated/seaborn.swarmplot.html), but can be plotted much more quickly because the noise on the x-dimension is random Normal.

The grey circles in the center of each column are the means for those classes.

`scprep.plot.jitter` API: https://scprep.readthedocs.io/en/stable/reference.html#scprep.plot.jitter

Jitterplot gallery: https://scprep.readthedocs.io/en/stable/examples/jitter.html

In [ ]:
# Get all the cells that are on the desired branch.
curr_branch = 'slingshot1'
mask = np.isfinite(metadata[curr_branch])

# Create Jitterplot
ax = scprep.plot.jitter(metadata['annotated_clusters'].loc[mask], metadata[curr_branch].loc[mask],
                        c=metadata['annotated_clusters'].loc[mask], legend_anchor=(1,1),
                        means_s=50,
                        figsize=(8,4))

# Neaten xticklabels
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
scprep.plot.utils.shift_ticklabels(ax.xaxis, dx=0.15)

In [ ]:
# Get all the cells that are on the desired branch.
curr_branch = 'slingshot2'
mask = np.isfinite(metadata[curr_branch])

# Create Jitterplot
ax = scprep.plot.jitter(metadata['annotated_clusters'].loc[mask], metadata[curr_branch].loc[mask],
                        c=metadata['annotated_clusters'].loc[mask], legend_anchor=(1,1),
                        means_s=50,
                        figsize=(8,4))

# Neaten xticklabels
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
scprep.plot.utils.shift_ticklabels(ax.xaxis, dx=0.15)

### Dicsussion

Each pair per table should:
1. Pick a different dimensionality reduction method (UMAP, t-SNE, or PCA).
2. Pick a different clustering method (see earleir exercises for code).
3. Rerun Slingshot
4. Compare results

In groups, discuss the following questions:
1. Do these orderings make sense given the cluster labels? Why?
2. How does choice of dimensionality reduction tool affect the output of Slingshot?
3. Does choice of dimensionality reduction tool or cluster method have more impact on output?

#### Breakpoint: if you're done, help those around you!

## Running Diffusion Pseudotime

In this section, we're going to use the diffusion pseudotime algorithm implemented in `scanpy`, a scRNA-seq analysis toolkit from Alex Wolf and the Theis Lab. The Theis lab were the inventors of the algorithm, so we're going to use their implementation to provide a comparison to Slingshot. As this MEF dataset is fairly simply, we don't expect to see huge differences between the algorithms. 

Because `scanpy` is a Python package, we can just use their code directly. The documentation for the `scanpy.tl.dpt` function can be found here: https://icb-scanpy.readthedocs-hosted.com/en/stable/api/scanpy.api.tl.dpt.html.

#### About the  `AnnData` object

All of the `scanpy` functions expect data to be organized in an `AnnData` object. The idea behind `AnnData` is to have a single container for all of the data and metadata associated with a single cell experiment. The `AnnData` class is useful and flexible. We don't use the `AnnData` class elsewhere in the workshop because we prefer to directly interact with Pandas `DataFrame` objects which are a more generic data container. The `DataFrame` is a useful data container for almost any kind of data one might work with, while `AnnData` is designed around gene expression data.

You very well may find that you prefer working with `AnnData` objects because of the handy single-cell functionality built into `scanpy`. If you want to learn more about them, check out the official `AnnData` documentation here: https://anndata.readthedocs.io.

### Running DPT

The workflow here is fairly simple. We're going to put the data in an `Anndata` container, run PCA, calculate NearestNeighbors, and DiffusionMaps. With that done, we are ready to run DPT.

In [ ]:
# Here we create an AnnData container
adata = anndata.AnnData(data)

# DPT requires a root cell, depending on the orientation of your
# plot, you should be able to use the cell with the highest 
# data_phate[:,0] coordinate
adata.uns['iroot'] = np.argmax(data_phate.iloc[:,0].values)

# Run PCA
scanpy.pp.pca(adata)

# Identify nearest neighbors
scanpy.pp.neighbors(adata)

# Create the diffusion map
scanpy.tl.diffmap(adata)

# Run Diffusion Pseudotime with 1 branching event
scanpy.tl.dpt(adata, n_branchings=1)

# Grab the output and store in our metadata DataFrame
metadata['dpt'] = adata.obs['dpt_pseudotime']
metadata['dpt_branch'] = adata.obs['dpt_groups'].astype(int)

### Plotting pseudotime and branch assignments

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,5))

ax = scprep.plot.scatter2d(data_phate, c=metadata['dpt'], 
                           ticks=None, label_prefix='PHATE',
                           title='Diffusion Pseudotime',
                           ax=axes[0])

scprep.plot.scatter2d(data_phate, c=metadata['dpt_branch'], 
                      ticks=None, label_prefix='PHATE',
                      title='DPT branches',
                      legend_anchor=(1,1),
                      ax=axes[1])

fig.tight_layout()

#### Breakpoint: if you're done, help those around you!

## Plotting clusters by pseudotime

In [ ]:
ax = scprep.plot.jitter(metadata['annotated_clusters'], metadata['dpt'], 
                        c=metadata['annotated_clusters'], legend_anchor=(1,1), figsize=(8,4))

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
scprep.plot.utils.shift_ticklabels(ax.xaxis, dx=0.15)

## Comparing DPT vs Slingshot

In [ ]:
# Get all the cells that are on the desired branch.
curr_branch = 'slingshot1'
mask = np.isfinite(metadata[curr_branch])

ax = scprep.plot.scatter(metadata[curr_branch].loc[mask], metadata['dpt'].loc[mask],
                         c=metadata['annotated_clusters'].loc[mask],
                         title='Slingshot vs. DPT - Branch 1',
                         legend_anchor=(1,1))

In [ ]:
# Get all the cells that are on the desired branch.
curr_branch = 'slingshot2'
mask = np.isfinite(metadata[curr_branch])

ax = scprep.plot.scatter(metadata[curr_branch].loc[mask], metadata['dpt'].loc[mask],
                         c=metadata['annotated_clusters'].loc[mask],
                         title='Slingshot vs. DPT - Branch 1',
                         legend_anchor=(1,1))

### Optional - Plotting genes by pseudotime

Try plotting genes like `Vim`, `Fn1`, `Ascl1`, `Myl1`, or `Stemn3` against either slingshot or diffusion pseudotime. 

### Discussion

In groups, discuss the following questions:

1. How well do the pseudotime orderings from Slingshot and DPT agree?
2. How could you quantify this?
3. Notice how there is a gap in pseudotime identified by both methods. Why do you think this happens? Provide both an algorithmic explanation (mathematically why is there a gap) and a biologically plausible explanation (what does this suggest about differentiation)?